In [4]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re

In [74]:
import psycopg2
import copy

In [32]:
from nltk.stem import WordNetLemmatizer 
from nltk import pos_tag, word_tokenize
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

In [51]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


In [203]:
def get_def_info(word, from_lang, to_lang, multisence_dict, print_ouput = False):
    multisence_dict[word] = {'lemma_branch':[], 'wordforms_branch':{}}
    definitions = []
    same_page = False
    for index in range(1,4):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word + "_" + str(index)
        if print_ouput:print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        for link in soup.find_all(["span","b"]):
    
            classes = link.get('class')
            if classes and "def" in classes:
                if link.text.strip() in definitions:
                    same_page = True
                    break
                if print_ouput: print("DEFINITION", link.text.strip())
                current_definition['definition'] = link.text.strip()
                definitions.append(link.text.strip())
                
            if classes and "def-body" in classes:
                if print_ouput:print("RUS_DEFIN", link.find("span", attrs = {'class':"trans"}).text.strip())
                current_definition['local_word'] = link.find("span", attrs = {'class':"trans"}).text.strip()
                current_definition['examples'] = []
                for ex in link.find_all("span", attrs = {'class':"eg"}):
                    current_definition['examples'].append(ex.text.strip())
                    if print_ouput:print("ENG_DEFIN_EX", ex.text.strip())
                if print_ouput:print(current_definition)
                multisence_dict[word]['lemma_branch'].append(current_definition)
                current_definition = {}
                if print_ouput:print("===")

        if same_page == True:
            break
        time.sleep(random.uniform(0.5,0.9))
sence_dict = {}      
get_def_info("book", "english","russian",sence_dict)


In [187]:
sence_dict['book']


{'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
   'local_word': 'книга',
   'examples': ['a book about animals']},
  {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
   'local_word': 'книжечка',
   'examples': []},
  {'definition': 'a set of pages fastened together in a cover and used for writing on',
   'local_word': 'записная книжка',
   'examples': ['an address book']},
  {'definition': 'to arrange to use or do something at a particular time in the future',
   'local_word': 'бронировать',
   'examples': ['to book a ticket/hotel room',
    "We've booked a trip to Spain for next month.",
    'Sorry, the hotel is fully booked (= has no more rooms).']},
  {'definition': 'to officially accuse someone of a crime',
   'local_word': 'заводить дело',
   'examples': ['Detectives booked him for resisting arrest.']},
  {'definition': 'If a sports official books you, they write an official record of some

In [213]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()

In [212]:
cursor.close()
conn.close()

In [233]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def nltk_lemmatize(word):
    pos = pos_tag(word_tokenize(word))[0][1]
    wordnet_pos = get_wordnet_pos(pos)
    if wordnet_pos:
        lemma = lemmatizer.lemmatize(word, pos = wordnet_pos)
    else:
        lemma = lemmatizer.lemmatize(word)
    return lemma

nltk_lemmatize("baddest")

'baddest'

In [215]:
def get_wordform_info_from_reverso(eng_wordform, from_lang, to_lang, print_output = False):
    wordform_localsencewords_examples_dict = {}
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + eng_wordform 
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    soup = bs(response.text, 'html.parser')
    first_string_passed = False
    translations_variants = []
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        if first_string_passed == True:
            sence_word = link.text.strip()
            if print_output:print(sence_word)
            wordform_localsencewords_examples_dict[sence_word] = []
            translations_variants.append(sence_word)
        first_string_passed = True
    time.sleep(random.uniform(0.4,1.2))
    for transl_word in translations_variants:
        time.sleep(random.uniform(0.4,2.9))
        url_example = "https://context.reverso.net/перевод/" + to_lang + "-" + from_lang + "/" + transl_word
        response_ex = requests.get(url_example, headers=header_main)
        soup_sub = bs(response_ex.text, 'html.parser')
        if print_output:
            print("========================================")
            print("\nEXAMPLES FOR SENCE ",transl_word,"\n" )
        for l in soup_sub.find_all('span', attrs = {'class':'text', 'lang':'en'}):
            example = l.text.strip()
            #print(example, word in example, type(l.text.strip()), word)
            regex = "\W" + eng_wordform + "\w{0,1}\W"
            find = re.findall(regex, example)
            if len(find) > 0 :
                wordform_localsencewords_examples_dict[transl_word].append(example)
                if print_output: print(example)
    return wordform_localsencewords_examples_dict
#w_locsence_ex_dict = get_wordform_info_from_reverso('book',"английский","русский", print_output = True )   
    

In [191]:
w_locsence_ex_dict

{'книга': ['Recently has published as a co-author with Ra-Ma the book entitled Internet protocols.',
  'The book aimed to sensitize children to the different aspects of poverty.',
  'The book is available on the OHCHR web site.',
  "It's Steve Owen's accounting book, with all the cooking.",
  'My love life is an open book and a short and nasty one.',
  "All he's got is writing stuff and this book.",
  'I mean, he has a book about all of this.',
  'So, Cleo, this book generating a lot of chatter.',
  'In a printed form the book is for the time being only available in German.',
  "It's not a modern-day book, it's a classic.",
  'A dirty book is not a substitute for a girlfriend.',
  "It's a proper book, it's not trash.",
  'And Mr. Griffin, I just have to say, I loved your book.',
  "It's a precious book of life that brings forth the blessings of Heaven.",
  'A book on the major achievements of the Joint Committee will be presented.',
  "This book is about finding your soul's purpose or 

In [235]:
def merge_context_vs_oxford_examples(eng_lemma, local_wordsence_examples_dict, general_dict, print_output = False):
    oxford_sensewords_list = []
    oxford_lemmas_examples_pointers_dict = {}
    for definition_element in general_dict[eng_lemma]['lemma_branch']:
        #if print_output: print(definition_element)
        oxford_sensewords_list.append(definition_element['local_word'])
        oxford_lemmas_examples_pointers_dict[definition_element['local_word']] = definition_element['examples']
    if print_output: print("WILL LOOK THROUGH LOCAL WORDFORMS", oxford_sensewords_list)
    oxford_sensewords_list = set(oxford_sensewords_list)
    for context_local_lemma in local_wordsence_examples_dict.keys():
        if print_output: print("\nNOWPROCESSING", context_local_lemma)
        orig_intersection_lemma = look_for_sublemma_word(context_local_lemma, oxford_sensewords_list)
        if not orig_intersection_lemma:
            if print_output: print(context_local_lemma, "\nno such lemma in orig dict")
            pass
        elif len(local_wordsence_examples_dict[context_local_lemma]) == 0:
            if print_output: print(context_local_lemma, "\ndoes not have examples with english wodform")
            pass
        else:
            #general_dict[eng_lemma]['lemma_branch'][context_local_lemma]['examples'].extend(local_wordsence_examples_dict[context_local_lemma][:3])
            if print_output:print(oxford_lemmas_examples_pointers_dict[orig_intersection_lemma])
            oxford_lemmas_examples_pointers_dict[orig_intersection_lemma].extend(local_wordsence_examples_dict[context_local_lemma][:3])
            if print_output:print("add",local_wordsence_examples_dict[context_local_lemma][:3] )
dct = copy.deepcopy(sence_dict)
merge_context_vs_oxford_examples("book",w_locsence_ex_dict, dct, print_output = True)        
dct


WILL LOOK THROUGH LOCAL WORDFORMS ['книга', 'книжечка', 'записная книжка', 'бронировать', 'заводить дело', 'штрафовать (в спорте)']

NOWPROCESSING книга
['a book about animals']
add ['Recently has published as a co-author with Ra-Ma the book entitled Internet protocols.', 'The book aimed to sensitize children to the different aspects of poverty.', 'The book is available on the OHCHR web site.']

NOWPROCESSING книжка
книжка 
no such lemma in orig dict

NOWPROCESSING справочник
справочник 
no such lemma in orig dict

NOWPROCESSING учебник
учебник 
no such lemma in orig dict

NOWPROCESSING журнал
журнал 
no such lemma in orig dict

NOWPROCESSING альбом
альбом 
no such lemma in orig dict

NOWPROCESSING тетрадь
тетрадь 
no such lemma in orig dict

NOWPROCESSING блокнот
блокнот 
no such lemma in orig dict

NOWPROCESSING книжный
книжный 
no such lemma in orig dict

NOWPROCESSING забронировать
['to book a ticket/hotel room', "We've booked a trip to Spain for next month.", 'Sorry, the hotel is 

{'book': {'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
    'local_word': 'книга',
    'examples': ['a book about animals',
     'Recently has published as a co-author with Ra-Ma the book entitled Internet protocols.',
     'The book aimed to sensitize children to the different aspects of poverty.',
     'The book is available on the OHCHR web site.']},
   {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
    'local_word': 'книжечка',
    'examples': ['Have you got an autograph book?',
     'Where is your book?',
     'You taking Little League book like you did last year?']},
   {'definition': 'a set of pages fastened together in a cover and used for writing on',
    'local_word': 'записная книжка',
    'examples': ['an address book']},
   {'definition': 'to arrange to use or do something at a particular time in the future',
    'local_word': 'бронировать',
    'examples': ['to book a ticket/hot

In [206]:

l = ['книга', 'книжечка', 'записная книжка', 'бронировать', 'заводить дело', 'штрафовать (в спорте)']
word = 'забронировать'
def look_for_sublemma_word(wordform, lemmas_list):
    if isinstance(lemmas_list, list) or isinstance(lemmas_list, set):
        for lemma in lemmas_list:
            if len(lemma.split()) == 1:
                if wordform in lemma or lemma in wordform:
                    #print(wordform)
                    return lemma
        return None
    elif isinstance(lemmas_list, str):
        if len(lemmas_list.split()) == 1:
            if wordform in lemmas_list or lemmas_list in wordform:
                return lemma
            else:
                return None
    else:
        return None #"no type" + str(type(lemmas_list))

look_for_sublemma_word(word, l) 
            

'бронировать'

In [193]:
def merge_locsence_ex_vs_lemma_structure(eng_wordform, eng_lemma, local_wordsence_examples_dict, general_dict, print_output = False):
    general_dict[eng_lemma]['wordforms_branch'][eng_wordform] = []
    oxford_sensewords_list = []
    for definition_element in general_dict[eng_lemma]['lemma_branch']:
        #if print_output: print(definition_element)
        oxford_sensewords_list.append(definition_element['local_word'])
    if print_output: print("WILL LOOK THROUGH LOCAL WORDFORMS", oxford_sensewords_list)
    oxford_sensewords_list = set(oxford_sensewords_list)
    successfull_merge_count = 0
    for local_wordform in local_wordsence_examples_dict.keys():
        p = morph.parse(local_wordform)[0]
        local_lemma = p.normal_form
        if print_output: print("\nNOWPROCESSING", local_wordform, local_lemma, look_for_sublemma_word (local_lemma, oxford_sensewords_list))
        if look_for_sublemma_word (local_wordform, oxford_sensewords_list):
            if print_output: print(local_wordform, "\nis original lemma itself")
            pass
        elif len(local_wordsence_examples_dict[local_wordform]) == 0:
            if print_output: print(local_wordform, "\ndoes not have examples with english wodform")
            pass
        else:
            if print_output: print(local_wordform, "\nno match but no limitations")
            local_wordform_dict = {"sence_wordform_local":local_wordform,"context":local_wordsence_examples_dict[local_wordform][:3]}
            if print_output: print("LOCAL_DICT_OBJECT",local_wordform_dict)
            general_dict[eng_lemma]['wordforms_branch'][eng_wordform].append(local_wordform_dict)
        """
        elif look_for_sublemma_word (local_lemma, oxford_sensewords_list) == True:
            if print_output: print(local_wordform, "\nlocal_wordform to local lemma or viceversa match")
            successfull_merge_count += 1
            local_wordform_dict = {"sence_wordform_local":local_wordform,
                                   "context":[]}
            for definition_element in general_dict[eng_lemma]['lemma_branch']:
                
                if look_for_sublemma_word (local_lemma, definition_element['local_word']):
                    local_wordform_examples = copy.deepcopy(definition_element['examples'])
                    local_wordform_dict['context'] = local_wordform_examples
                    try:
                        local_wordform_dict['context'].extend(local_wordsence_examples_dict[local_wordform][:3])
                        definition_element['examples'].extend(local_wordsence_examples_dict[local_wordform][:3])
                    except:
                        local_wordform_dict['context'].extend(local_wordsence_examples_dict[definition_element['local_word']][:3])
                        definition_element['examples'].extend(local_wordsence_examples_dict[definition_element['local_word']][:3])
            general_dict[eng_lemma]['wordforms_branch'][eng_wordform].append(local_wordform_dict)
            if print_output: print("LOCAL_DICT_OBJECT",local_wordform_dict)
        """
        
    if print_output:
        if successfull_merge_count == 0: print("NO LOCAL INTERSECTIONS FOR WORDWORM", eng_wordform)
        
        
merge_locsence_ex_vs_lemma_structure("booking","book",w_locsence_ex_dict, big_diCKt, print_output = True)        
big_diCKt

WILL LOOK THROUGH LOCAL WORDFORMS ['книга', 'книжечка', 'записная книжка', 'бронировать', 'заводить дело', 'штрафовать (в спорте)']

NOWPROCESSING книга книга книга
книга 
is original lemma itself

NOWPROCESSING книжка книжка записная книжка
книжка 
is original lemma itself

NOWPROCESSING справочник справочник None
справочник 
does not have examples with english wodform

NOWPROCESSING учебник учебник None
учебник 
no match but no limitations
LOCAL_DICT_OBJECT {'sence_wordform_local': 'учебник', 'context': ['The book is directed to teachers of the Roma language and meant for self-education.', 'I flipped through my geometry book in the john and totally figured out angles.', "You've been reading Tommy's geography book again..."]}

NOWPROCESSING журнал журнал None
журнал 
no match but no limitations
LOCAL_DICT_OBJECT {'sence_wordform_local': 'журнал', 'context': ['All the books say to keep a sleep journal.']}

NOWPROCESSING альбом альбом None
альбом 
does not have examples with english wod

{'book': {'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
    'local_word': 'книга',
    'examples': ['a book about animals',
     "It's not easy to get books in English around here.",
     'She sent the companion into Reading to exchange some library books, and took some sleeping pills.',
     'So there must be a reason why they used two books.']},
   {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
    'local_word': 'книжечка',
    'examples': []},
   {'definition': 'a set of pages fastened together in a cover and used for writing on',
    'local_word': 'записная книжка',
    'examples': ['an address book',
     'I read one of your books, by the way.',
     "Apparently he read a couple books in the '80s.",
     'I was so nervous with my mom, think I put like ten books in there.',
     "That's what you get in children's books.",
     'Do you want to be in the good books or the bad books?',
     

In [238]:
def process_lemma_and_forms(word, dictionary, show_words = False):
    get_def_info(word, "english","russian",dictionary)
    lemma_sence_ex_dict = get_wordform_info_from_reverso(word,"английский","русский", print_output = False )
    merge_context_vs_oxford_examples(word,lemma_sence_ex_dict, dictionary, print_output = False) 
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not req_res:
        if show_words == True: print(word, " is not in db")
        return 0
    else:
        #print(a[0])
        word_id = req_res[0]
        if show_words == True: print(req_res[2])
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    for wordform in cursor:
        if show_words == True: print(wordform[0])
        wordform_lemma = nltk_lemmatize(wordform[0])
        if wordform_lemma == word:
            if show_words == True: print("will process", wordform[0])
            local_wordforms_sence_examples_dict = get_wordform_info_from_reverso(wordform[0],"английский", "русский")  
            merge_locsence_ex_vs_lemma_structure(wordform[0],word, local_wordforms_sence_examples_dict,dictionary ,
                                                                                 print_output= False)
        else:
            if show_words == True: print("will not process", wordform[0], "lemma is",wordform_lemma )
        #get_def_info(wordform[0], "english","russian",dictionary,is_word_form = True, original_word = word )
        
big_diCKt = {}        
process_lemma_and_forms("book",big_diCKt)
big_diCKt

{'book': {'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
    'local_word': 'книга',
    'examples': ['a book about animals',
     'Recently has published as a co-author with Ra-Ma the book entitled Internet protocols.',
     'The book aimed to sensitize children to the different aspects of poverty.',
     'The book is available on the OHCHR web site.']},
   {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
    'local_word': 'книжечка',
    'examples': ['Have you got an autograph book?',
     'Where is your book?',
     'You taking Little League book like you did last year?']},
   {'definition': 'a set of pages fastened together in a cover and used for writing on',
    'local_word': 'записная книжка',
    'examples': ['an address book']},
   {'definition': 'to arrange to use or do something at a particular time in the future',
    'local_word': 'бронировать',
    'examples': ['to book a ticket/hot

In [217]:
import json
with open("one_word_transalte.json", "w") as f:
    json.dump(big_diCKt, f, ensure_ascii=False, indent = 4)

In [216]:
big_diCKt

{'book': {'lemma_branch': [{'definition': 'a set of pages fastened together in a cover for people to read',
    'local_word': 'книга',
    'examples': ['a book about animals',
     'Recently has published as a co-author with Ra-Ma the book entitled Internet protocols.',
     'The book aimed to sensitize children to the different aspects of poverty.',
     'The book is available on the OHCHR web site.']},
   {'definition': 'a set of stamps, tickets, etc that are fastened together inside a cover',
    'local_word': 'книжечка',
    'examples': ['Have you got an autograph book?',
     'Where is your book?',
     'You taking Little League book like you did last year?']},
   {'definition': 'a set of pages fastened together in a cover and used for writing on',
    'local_word': 'записная книжка',
    'examples': ['an address book']},
   {'definition': 'to arrange to use or do something at a particular time in the future',
    'local_word': 'бронировать',
    'examples': ['to book a ticket/hot

In [229]:
import pandas as pd
from tqdm import tqdm

In [226]:
test_words_db = pd.read_csv("test_words.csv")
test_words = list(test_words_db['word'])

In [239]:
global_dict = {}
for word in tqdm(test_words):
    process_lemma_and_forms(word.lower(), global_dict)


  6%|▌         | 12/200 [08:02<2:48:46, 53.86s/it]

KeyboardInterrupt: 

In [240]:
with open("big_dict_attempt.json", "w") as f:
    json.dump(global_dict, f, ensure_ascii=False, indent = 4)